### Data Visualizaiton
Plotting can be challenging with a wide dataset, with 432 columns.<br>
For that reason, we created a few reusable functions to simply<br>
the process.<br>

| Function | Description |
|:--- | :--- |
| `drug_test_df` | This function uses the following workflow to subset data:
Filter data, select columns, rename columns, sum columns, create series|
|                | Filter data, select columns, rename columns, sum columns, create series    
| `plot_drug_test` | Plot the patient's data |        

We will explore the dataset to answer the following research questions:
1. What is the trajectory of recovery for patients by medication
2. What is the pattern for recovery in each medication group

In [1]:
# import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import helper 
import warnings
warnings.filterwarnings('ignore')

# load the data
data = pd.read_csv('../data/merged_data.csv')

data.shape

(1917, 431)

### Build a Dataframe with Time Series for Plotting

In [2]:
# call the helper function to create test dataframe
methadone, buprenorphine = helper.test_dataframes(data)

AttributeError: module 'helper' has no attribute 'test_dataframes'

In [ ]:
# set figure size 
fig = plt.figure(figsize=(12, 14))

# use barpl

In [ ]:
methadone[:1]

In [ ]:
buprenorphine[:1]

In [ ]:
def plot_tests_groups(df,medication):
    """
    function accepts a dataframe and medication name
    to easily plot meaningful data in to show trajaectory
    of the patients recovery, compared to other groups.

    """
    if medication == 'methadone':
        # create barplot for methadone group
        sns.barplot(x='time', y='value', hue='variable', data=df)
    else:
        # create barplot for buprenorphine group
        sns.barplot(x='time', y='value', hue='variable', data=df)

    if medication == 'methadone':
        # use lineplot to plot methadone tests
        plt.plot(np.array(df.index), np.array(df['met']), color='blue', label='Methadone', linewidth=2.5);
    else:
        # use lineplot to plot buprenorphine tests
        plt.plot(np.array(df.index), np.array(df['bup']), color='red', label='Buprenorphine', linewidth=2.5);

In [ ]:
# set figure size
fig = plt.figure(figsize=(12, 4))

# use barplot to plot all tests
sns.barplot(x=np.array(tests.index), y='all', data=tests, color='darkgray')

# use lineplot to plot methadone tests
plt.plot(np.array(tests.index), np.array(tests['met']), color='blue', label='Methadone', linewidth=2.5);

# use lineplot to plot buprenorphine tests
plt.plot(np.array(tests.index), np.array(tests['bup']), color='red', label='Buprenorphine', linewidth=2.5);

# Add labels and title
plt.title('Positive Opiate Tests by Week and Medication Type')
plt.ylabel('Count of Positive Opiate Tests')
plt.xlabel('Week of Treatment')
plt.legend();

In [ ]:
# set figure size
fig = plt.figure(figsize=(12, 6))

# use barplot to plot all tests
sns.barplot(x=np.array(methadone.index), y='all', data=methadone, color='darkgray', alpha=0.5, edgecolor='black')

# use lineplot to plot methadone tests for patients who responded to treatment
plt.plot(np.array(methadone.index), np.array(methadone['met_r']), color='blue', label='Responder', linewidth=2.5);

# use lineplot to plot methadone tests for patients who did not respond to treatment
plt.plot(np.array(methadone.index), np.array(methadone['met_nr']), color='red', label='Non-Responder', linewidth=2.5);

# Add labels and title
plt.title('Positive Opiate Tests for Methadone Patients by Week and Response')
plt.ylabel('Count of Positive Opiate Tests')
plt.xlabel('Week of Treatment')
plt.legend(['responder', 'non-responder'])
plt.legend();


In [ ]:
# set figure size
fig = plt.figure(figsize=(12, 6))

# use barplot to plot all tests
sns.barplot(x=np.array(buprenorphine.index), y='all', data=buprenorphine, color='darkgray', alpha=0.5, edgecolor='black')

# use lineplot to plot buprenorphine tests for patients who responded to treatment
plt.plot(np.array(buprenorphine.index), np.array(buprenorphine['bup_r']), color='blue', label='Responder', linewidth=2.5);

# use lineplot to plot buprenorphine tests for patients who did not respond to treatment
plt.plot(np.array(buprenorphine.index), np.array(buprenorphine['bup_nr']), color='red', label='Non-Responder', linewidth=2.5);

# Add labels and title
plt.title('Positive Opiate Tests for Buprenorphine Patients by Week and Response')
plt.ylabel('Count of Positive Opiate Tests')
plt.xlabel('Week of Treatment')
plt.legend();

In [ ]:
# create df with bup responders and met responders
responder = pd.concat([methadone[['met_r']],buprenorphine[['bup_r']]], axis=1)

responder['all'] = responder['met_r'] + responder['bup_r']

responder

In [ ]:
# set figure size
fig = plt.figure(figsize=(12, 6))

# use barplot to plot all tests
sns.barplot(x=np.array(responder.index), y='all', data=responder, color='darkgray', alpha=0.5, edgecolor='black')

# use lineplot to plot buprenorphine tests for patients who responded to treatment
plt.plot(np.array(responder.index), np.array(responder['bup_r']), color='blue', label='Buprenorphine', linewidth=2.5);

# use lineplot to plot methadone tests for patients who responded to treatment
plt.plot(np.array(responder.index), np.array(responder['met_r']), color='red', label='Methadone', linewidth=2.5);

# Add labels and title
plt.title('Positive Opiate Tests for Responders by Week and Medication')
plt.ylabel('Count of Positive Opiate Tests')
plt.xlabel('Week of Treatment')
plt.legend();


In [ ]:
# create df with bup non-responders and met non-responders
non_responder = pd.concat([methadone[['met_nr']],buprenorphine[['bup_nr']]], axis=1)

non_responder['all'] = non_responder['met_nr'] + non_responder['bup_nr']

non_responder

In [ ]:
# set figure size
fig = plt.figure(figsize=(12, 6))

# use barplot to plot all tests
sns.barplot(x=np.array(non_responder.index), y='all', data=non_responder, color='darkgray', alpha=0.5, edgecolor='black')

# use lineplot to plot buprenorphine tests for patients who did not respond to treatment
plt.plot(np.array(non_responder.index), np.array(non_responder['bup_nr']), color='blue', label='Buprenorphine', linewidth=2.5);

# use lineplot to plot methadone tests for patients who did not respond to treatment
plt.plot(np.array(non_responder.index), np.array(non_responder['met_nr']), color='red', label='Methadone', linewidth=2.5);

# Add labels and title
plt.title('Positive Opiate Tests for Non-Responders by Week and Medication')
plt.ylabel('Count of Positive Opiate Tests')
plt.xlabel('Week of Treatment')
plt.legend()
plt.show();

# create df with bup non-responders and met responders